In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
#imdb = load_from_disk("../data/imdb")
imdb = load_dataset('artemis13fowl/imdb')

Using custom data configuration artemis13fowl--imdb-f63738dec0d5e230
Reusing dataset parquet (/home/jasko/.cache/huggingface/datasets/parquet/artemis13fowl--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
imdb

DatasetDict({
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
    attack_eval_truncated: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [4]:
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
max_sequence_length = 128
batch_size = 32
eval_steps = 100
learning_rate=2e-05
num_train_epochs=5
output_dir = "../output/"
model_dir = "../models/"
early_stopping_patience = 10

In [6]:
def tokenize_function(example):
    return tokenizer(example["text"],  truncation=True, padding="max_length", max_length=max_sequence_length )


tokenized_datasets = imdb.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [7]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", ])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [8]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
# )
# dev_dataloader = DataLoader(
#     tokenized_datasets["dev"], batch_size=16, collate_fn=data_collator
# )

In [9]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [10]:
from transformers import AutoModelForSequenceClassification, RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [11]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

In [12]:
# def compute_metrics(eval_preds):
#     metric = load_metric("imdb)
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [13]:

# training_args = TrainingArguments('bert-base-uncased-imdb', 
#                                   learning_rate=2e-05,
#                                   per_device_train_batch_size =batch_size, 
#                                   per_device_eval_batch_size =batch_size,

#                           )

training_args = TrainingArguments(
   output_dir+"roberta-base-imdb",
   evaluation_strategy ='steps',
   eval_steps = eval_steps , # Evaluation and Save happens every eval_steps steps
   save_total_limit = 1, # Only last  model is saved. Older ones are deleted.
   learning_rate=learning_rate,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   num_train_epochs=num_train_epochs,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

In [14]:
from transformers import Trainer, EarlyStoppingCallback

In [15]:

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
    compute_metrics=compute_metrics,
)


In [16]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Runtime,Samples Per Second
100,No log,0.311536,0.876000,0.841328,0.923077,0.880309,5.089100,196.497000
200,No log,0.271896,0.890000,0.893443,0.882591,0.887984,5.030500,198.787000
300,No log,0.270422,0.881000,0.847866,0.925101,0.884802,5.074400,197.066000
400,No log,0.305204,0.889000,0.840142,0.957490,0.894986,5.055600,197.801000
500,0.321600,0.266000,0.890000,0.849091,0.945344,0.894636,5.065700,197.406000
600,0.321600,0.238922,0.899000,0.906832,0.886640,0.896622,5.085900,196.623000
700,0.321600,0.253384,0.906000,0.925532,0.880567,0.902490,5.172100,193.345000
800,0.321600,0.305635,0.898000,0.881323,0.917004,0.898810,5.075900,197.010000
900,0.321600,0.312137,0.897000,0.953596,0.831984,0.888649,5.075000,197.043000
1000,0.227200,0.271154,0.891000,0.924945,0.848178,0.884900,5.095500,196.253000


TrainOutput(global_step=2600, training_loss=0.19386381516089807, metrics={'train_runtime': 1576.8122, 'train_samples_per_second': 2.48, 'total_flos': 7957762519879680, 'epoch': 3.32})

In [17]:
model.save_pretrained(model_dir+"roberta-base-imdb")

In [18]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(24000, 2) (24000,)


In [19]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)

In [20]:
print(classification_report(predictions.label_ids, preds))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90     11994
           1       0.87      0.95      0.91     12006

    accuracy                           0.91     24000
   macro avg       0.91      0.91      0.91     24000
weighted avg       0.91      0.91      0.91     24000

